In [ ]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
import getpass
from dotenv import load_dotenv
from groq import Groq

load_dotenv()

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

client = Groq(api_key=os.environ["GROQ_API_KEY"])

MODEL_CONFIG = {
    "technical": {
        "model": "llama-3.3-70b-versatile",
        "system_prompt": (
            "You are a Senior Technical Support Engineer. "
            "You are rigorous, code-focused, and precise. "
            "Provide technical solutions, code snippets, and debugging steps. "
            "Do not be overly chatty; focus on the fix."
        )
    },
    "billing": {
        "model": "llama-3.3-70b-versatile",
        "system_prompt": (
            "You are a Customer Success Manager specializing in Billing. "
            "You are empathetic, polite, and policy-driven. "
            "Explain charges clearly and guide users through refund processes. "
            "Apologize for any inconvenience."
        )
    },
    "general": {
        "model": "llama-3.3-70b-versatile",
        "system_prompt": (
            "You are a helpful general assistant. "
            "Answer questions politely and concisely. "
            "If you don't know the answer, admit it."
        )
    },
    "tool": {
        "type": "function",
        "handler": "get_bitcoin_price"
    }
}

def route_prompt(user_input):
    """
    Decides which expert should handle the query.
    Returns: 'technical', 'billing', 'general', or 'tool'
    """
    completion = client.chat.completions.create(
        # UPDATED: Using 'llama-3.1-8b-instant' for the router as it is faster and cheaper
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a classification system. "
                    "Analyze the user's input and classify it into one of these categories:\n"
                    "- technical (for code errors, bugs, software issues)\n"
                    "- billing (for refunds, payments, subscriptions)\n"
                    "- tool (specifically for questions about the price of Bitcoin)\n"
                    "- general (for greetings or anything else)\n\n"
                    "Output ONLY the category name (lowercase). Do not explain."
                )
            },
            {
                "role": "user",
                "content": user_input
            }
        ],
        temperature=0.0, # Strict determinism
        max_tokens=10
    )

    category = completion.choices[0].message.content.strip().lower()
    return category

def get_bitcoin_price():
    return "The current mock price of Bitcoin is $95,432.10"

def process_request(user_input):
    print(f"\nUser Query: \"{user_input}\"")

    category = route_prompt(user_input)

    if category not in MODEL_CONFIG:
        category = "general"

    print(f"--> [Router] Selected Expert: {category.upper()}")

    if category == "tool":
        result = get_bitcoin_price()
        print(f"--> [System] {result}")
        return result

    expert_config = MODEL_CONFIG[category]

    completion = client.chat.completions.create(
        model=expert_config["model"],
        messages=[
            {"role": "system", "content": expert_config["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        temperature=0.7
    )

    response = completion.choices[0].message.content
    print(f"--> [Expert Response]:\n{response}\n")
    return response

if __name__ == "__main__":
    process_request("My python script is throwing an IndexError on line 5.")
    process_request("I was charged twice for my subscription this month. I want a refund.")
    process_request("Hi, how are you today?")
    process_request("What is the current price of Bitcoin?")


User Query: "My python script is throwing an IndexError on line 5."
--> [Router] Selected Expert: TECHNICAL
--> [Expert Response]:
To assist with the `IndexError` on line 5 of your Python script, I'll need more information. 

1. **Provide the code**: Please share the relevant code snippet, specifically lines 1-10, to help identify the issue.
2. **Error message**: Share the full error message you're receiving, as it may contain valuable information about the index that's causing the issue.
3. **Expected behavior**: Describe what you expect the script to do and what input you're providing.

With this information, I can provide a more precise solution, including code snippets and debugging steps to resolve the `IndexError`.


User Query: "I was charged twice for my subscription this month. I want a refund."
--> [Router] Selected Expert: BILLING
--> [Expert Response]:
I'm so sorry to hear that you were charged twice for your subscription this month. I can understand how frustrating that m